<a href="https://colab.research.google.com/github/alexisjihyeross/cs287_causality_project/blob/master/CS287_Bert_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install pytorch-pretrained-bert

    100% |████████████████████████████████| 122kB 8.6MB/s 


In [0]:
!wget https://gist.githubusercontent.com/W4ngatang/60c2bdb54d156a41194446737ce03e2e/raw/17b8dd0d724281ed7c3b2aeeda662b92809aadd5/download_glue_data.py
    
!python download_glue_data.py --data_dir glue_data --tasks MNLI # select any number of MNLI,SNLI,QNLI,WNLI

--2019-04-12 16:23:37--  https://gist.githubusercontent.com/W4ngatang/60c2bdb54d156a41194446737ce03e2e/raw/17b8dd0d724281ed7c3b2aeeda662b92809aadd5/download_glue_data.py
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8225 (8.0K) [text/plain]
Saving to: ‘download_glue_data.py’

download_glue_data. 100%[===================>]   8.03K  --.-KB/s    in 0s      

2019-04-12 16:23:37 (102 MB/s) - ‘download_glue_data.py’ saved [8225/8225]

	Completed!


In [0]:
!wget -P /content/glue_data/MNLI/ https://github.com/alexisjihyeross/cs287_causality_project/blob/master/neg_test_matched.tsv
!wget -P /content/glue_data/MNLI/ https://github.com/alexisjihyeross/cs287_causality_project/blob/master/neg_test_mismatched.tsv

--2019-04-12 16:23:57--  https://github.com/alexisjihyeross/cs287_causality_project/blob/master/neg_test_matched.tsv
Resolving github.com (github.com)... 140.82.118.3, 140.82.118.4
Connecting to github.com (github.com)|140.82.118.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘/content/glue_data/MNLI/neg_test_matched.tsv’

neg_test_matched.ts     [ <=>                ]  54.72K   295KB/s    in 0.2s    

2019-04-12 16:23:58 (295 KB/s) - ‘/content/glue_data/MNLI/neg_test_matched.tsv’ saved [56029]

--2019-04-12 16:23:59--  https://github.com/alexisjihyeross/cs287_causality_project/blob/master/neg_test_mismatched.tsv
Resolving github.com (github.com)... 140.82.118.3, 140.82.118.4
Connecting to github.com (github.com)|140.82.118.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘/content/glue_data/MNLI/neg_test_mismatched.tsv’

neg_test_mismatched     [ <=>           

In [0]:
import os
import csv
import sys
import collections
from tqdm import tqdm

In [0]:
from pytorch_pretrained_bert import BertConfig, BertTokenizer, BertModel
from pytorch_pretrained_bert.modeling import BertPreTrainedModel
from pytorch_pretrained_bert.optimization import BertAdam
import torch
import torch.nn as nn
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader, TensorDataset

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [0]:
InputExample = collections.namedtuple('InputExample', ['guid', 'text_a', 'text_b', 'label'])
InputFeatures = collections.namedtuple('InputFeatures', ['input_ids', 'input_mask', 'segment_ids', 'label_id'])

def convert_examples_to_features(examples, label_list, max_seq_length, tokenizer):
    features = []
    for (ex_index, example) in enumerate(examples):
        if ex_index % 10000 == 0:
            print("Writing example %d of %d" % (ex_index, len(examples)))

        tokens_a = tokenizer.tokenize(example.text_a)
        tokens_b = tokenizer.tokenize(example.text_b) if example.text_b else None

        # truncate sequence pairs, longest one first, accounting for [cls] and [sep]
        if example.text_b:
            while len(tokens_a) + len(tokens_b) > max_seq_length - 3:
                max(tokens_a, tokens_b, key=len).pop()
        elif len(tokens_a) > max_seq_length - 2:
            tokens_a = tokens_a[:(max_seq_length - 2)]
                
        # segment_ids indicate token in first or second seq, w/ embedding vectors
        # learned for them during pre-training, added to position embedding
        tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
        
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens += tokens_b + ["[SEP]"]
            segment_ids += [1] * (len(tokens_b) + 1)

        input_ids = tokenizer.convert_tokens_to_ids(tokens)
        
        input_mask = [1] * len(input_ids) # Only attend to real tokens

        seqs = [tokens, input_ids, input_mask, segment_ids]
        
        padding = [0] * (max_seq_length - len(input_ids))
        for seq in seqs[1:]:
            seq += padding
            assert len(seq) == max_seq_length

        label_map = {label: i for i, label in enumerate(label_list)}
        label_id = label_map[example.label]
        
        if ex_index < 3:
            print("*** Example ***")
            print("guid: %s" % (example.guid))
            for n, d in zip(['tokens', 'input_ids', 'input_mask', 'segment_ids'], [tokens, input_ids, input_mask, segment_ids]):
                print(f'{n}: {" ".join([str(x) for x in d])}')
            print("label: %s (id = %d)" % (example.label, label_id))

        features.append(InputFeatures(input_ids, input_mask, segment_ids, label_id))
    return features

In [0]:
class MnliProcessor:
    """Processor for the MultiNLI data set (GLUE version)."""
    @classmethod
    def _read_tsv(cls, input_file, quotechar=None):
        """Reads a tab separated value file."""
        with open(input_file, "r") as f:
            reader = csv.reader(f, delimiter="\t", quotechar=quotechar)
            lines = []
            for line in reader:
                if sys.version_info[0] == 2:
                    line = list(unicode(cell, 'utf-8') for cell in line)
                lines.append(line)
            return lines
        
    def get_labels(self):
        """Gets the list of labels for this data set."""
        return ["contradiction", "entailment", "neutral"]

    def _create_examples(self, lines, set_type):
        """Creates examples for the training and dev sets."""
        examples = []
        for (i, line) in enumerate(lines):
            if i == 0:
                continue
            guid = "%s-%s" % (set_type, line[0])
            text_a = line[8]
            text_b = line[9]
            label = line[-1]
            examples.append(InputExample(guid, text_a, text_b, label))
        return examples
    
    def get_dataloader(self, data_dir, data_file, tokenizer, batch_size=10, max_seq_len=70):
        if data_file not in ['dev_mismatched', 'dev_matched', 'test_matched', 'test_mismatched', 'neg_test_matched', 'neg_test_mismatched', 'train']:
            raise KeyError(f'Invalid data file {data_file}')
            
        data = self._read_tsv(os.path.join(data_dir, f"{data_file}.tsv"))
        examples = self._create_examples(data, data_file)
        labels = self.get_labels()
        
        train_feats = convert_examples_to_features(examples, labels, max_seq_len, tokenizer)
        dataset = [torch.tensor(d, dtype=torch.long) for d in zip(*train_feats)]
    
        return DataLoader(TensorDataset(*dataset), batch_size=batch_size)

In [0]:
class BertForSequenceClassification(BertPreTrainedModel):
    """BERT model w/ pooled output to linear layer 
    Params:
        `config`: a BertConfig instance with the config to build a new model
        `num_labels`: number of classes for classifier. Default = 2
    Inputs:
        `input_ids`: [batch_size, seq_length] of word token indices
        `token_type_ids`: optional [batch_size, seq_length] with [0, 1] for sentence [A, B] tokens
        `attention_mask`: optional [batch_size, seq_length] with [0, 1] to mask out attention on padding
        `labels`: optional [batch_size] of class output [0, ..., num_labels]
    Outputs:
        if `labels` is not `None`: CrossEntropy loss of output with labels.
        if `labels` is `None`: Class logits [batch_size, num_labels]
    """
    def __init__(self, config, num_labels):
        super(BertForSequenceClassification, self).__init__(config)
        self.num_labels = num_labels
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, num_labels)
        self.apply(self.init_bert_weights)

    def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None, modification=None):
        _, pooled_output = self.bert(input_ids, token_type_ids, attention_mask, output_all_encoded_layers=False)
        
        if modification:
            pooled_output = modification(pooled_output)
        
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        if labels is not None:
            loss_fct = CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            return loss, pooled_output
        else:
            return logits, pooled_output

In [0]:
BERT_SIZE = 'base'  # or 'large'
BERT_CASED = False
DATA_DIR = 'glue_data/MNLI'
CACHE_DIR = 'cache'
MODEL = f'bert-{BERT_SIZE}-{"cased" if BERT_CASED else "uncased"}'


tokenizer = BertTokenizer.from_pretrained(MODEL, do_lower_case=not BERT_CASED)

processor = MnliProcessor()

num_labels = len(processor.get_labels())

model = BertForSequenceClassification.from_pretrained(MODEL, cache_dir=CACHE_DIR, num_labels=num_labels)

100%|██████████| 407873900/407873900 [00:15<00:00, 25942253.66B/s]


In [0]:
train_dataloader = processor.get_dataloader(DATA_DIR, 'train', tokenizer, max_seq_len=70)

Writing example 0 of 392702
*** Example ***
guid: train-0
tokens: [CLS] conceptual ##ly cream ski ##mming has two basic dimensions - product and geography . [SEP] product and geography are what make cream ski ##mming work . [SEP]
input_ids: 101 17158 2135 6949 8301 25057 2038 2048 3937 9646 1011 4031 1998 10505 1012 102 4031 1998 10505 2024 2054 2191 6949 8301 25057 2147 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
label: neutral (id = 2)
*** Example ***
guid: train-1
tokens: [CLS] you know during the season and i guess at at your level uh you lose them to the next level if if they decide to recall the the parent team the braves decide to 

In [0]:
def train(model, dataloader, lr=5e-5, warmup=0.1, num_epochs=2, device='cuda', finetune=False):
    if not finetune:
        for param in model.bert.parameters():
            param.requires_grad = False
            
    loss_fct = CrossEntropyLoss()
    
    batch_size = dataloader.batch_size
    
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    params = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]

    optimizer = BertAdam(params, lr=lr, warmup=warmup, t_total=len(dataloader))

    model.to(device)
    
    model.train()
    
    for epoch in range(num_epochs):
        for batch in tqdm(train_dataloader):
            batch = tuple(t.to(device) for t in batch)
            input_ids, input_mask, segment_ids, label_ids = batch

            optimizer.zero_grad()

            logits, _ = model(input_ids, segment_ids, input_mask, labels=None)

            loss = loss_fct(logits.view(-1, num_labels), label_ids.view(-1))
            loss.backward()

            optimizer.step()

In [0]:
train(model, train_dataloader, num_epochs=1)

  0%|          | 17/39271 [00:02<1:11:26,  9.16it/s]